## Escenario 2: Este notebook simula un entorno de trabajo colaborativo donde un equipo de machine learning utiliza un servidor MLflow compartido para experimentar y registrar modelos.

MLflow setup:
- tracking server: con server local
- backend store: sqlite database
- artifacts store: sistema de archivos local
- Model Registry: Habilitado para gestión de modelos

Los experimentos pueden ser explorados usando el tracking server.

Antes de ejecutar este notebook, es necesario iniciar el servidor MLflow:

`mlflow server --backend-store-uri sqlite:///backend.db`

## Configuracion del tracking URI

Conecta el cliente MLflow al servidor local en el puerto 5000.


In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

Confirma que la conexión al servidor se estableció correctamente.

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


## Listado de Experimentos Disponibles

In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1755746272283, experiment_id='0', last_update_time=1755746272283, lifecycle_stage='active', name='Default', tags={}>]

## Ejecución del Experimento

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/08/20 22:18:33 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/08/20 22:18:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/20 22:18:35 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/08/20 22:18:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'mlflow-artifacts:/1/3025a45f2a324566b253a6b1423d51c6/artifacts'
🏃 View run serious-asp-645 at: http://127.0.0.1:5000/#/experiments/1/runs/3025a45f2a324566b253a6b1423d51c6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


## Resultados de la Ejecución

La ejecución genera:
- Un URI único para los artifacts
- Enlaces directos al servidor MLflow para visualizar:
    - La ejecución específica
    - El experimento completo

Ahora muestra dos experimentos:
- "Default" (creado automáticamente)
- "my-experiment-1" (nuestro experimento

In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1755746313659, experiment_id='1', last_update_time=1755746313659, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1755746272283, experiment_id='0', last_update_time=1755746272283, lifecycle_stage='active', name='Default', tags={}>]

### Interactuando con el model registry

Crea un cliente para interactuar con el servidor MLflow.

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

Inicialmente retorna una lista vacía, indicando que no hay modelos registrados.

In [7]:
client.search_registered_models()

[]

## Model Registry

1. Obtención del Run ID: Extrae el identificador de la ejecución más reciente
2. Registro del modelo: Crea una entrada en el model registry con el nombre 'iris-classifier'
3. Versionado: Automáticamente asigna la versión 1 al modelo

In [8]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/08/20 22:19:09 WARNING mlflow.tracking._model_registry.fluent: Run with id 3025a45f2a324566b253a6b1423d51c6 has no artifacts at artifact path 'models', registering model based on models:/m-123e4dbb070b4588934d29cfabc1e6fe instead
2025/08/20 22:19:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1755746349121, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1755746349121, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='3025a45f2a324566b253a6b1423d51c6', run_link='', source='models:/m-123e4dbb070b4588934d29cfabc1e6fe', status='READY', status_message=None, tags={}, user_id='', version='1'>

## **Model Registry vs Tracking: Diferencias Clave**

### **MLflow Tracking**
- **Propósito**: Seguimiento de experimentos y ejecuciones individuales
- **Almacena**: Parámetros, métricas, artifacts y metadatos de cada run
- **Uso**: Durante el desarrollo y experimentación
- **Organización**: Por experimentos y ejecuciones
- **Acceso**: A través de la API de MLflow o interfaz web

### **Model Registry**
- **Propósito**: Gestión del ciclo de vida completo de modelos en producción
- **Almacena**: Versiones de modelos, stages (Staging, Production), transiciones
- **Uso**: Para despliegue, versionado y gestión operacional
- **Organización**: Por nombre de modelo y versiones
- **Acceso**: A través del Model Registry API


## **Ventaja Principal del Model Registry**

**El Model Registry va más allá del tracking al proporcionar:**

1. **Versionado de Modelos**: Mantiene un historial completo de todas las versiones
2. **Stages de Despliegue**: Controla qué versión está en Staging vs Production
3. **Aprobaciones**: Permite flujos de trabajo para promocionar modelos
4. **Trazabilidad**: Conecta cada versión con su experimento original
5. **Operaciones**: Facilita rollbacks, comparaciones y auditorías

**En resumen**: Tracking es para experimentar, Registry es para producir y mantener modelos en el mundo real.

-----------

## **Diferencias Clave con el Escenario 1**

### **Arquitectura**
- **Escenario 1**: MLflow local con archivos
- **Escenario 2**: Servidor MLflow con base de datos SQLite

### **Colaboración**
- **Escenario 1**: Uso individual
- **Escenario 2**: Múltiples usuarios pueden acceder al mismo servidor

### **Persistencia**
- **Escenario 1**: Datos almacenados en archivos locales
- **Escenario 2**: Metadatos en base de datos SQLite, artifacts en sistema de archivos

### **Model Registry**
- **Escenario 1**: No disponible
- **Escenario 2**: Completamente funcional para gestión de modelos

---

## **Ventajas del Escenario 2**

1. **Colaboración**: Múltiples científicos de datos pueden trabajar en el mismo servidor
2. **Persistencia**: Los metadatos se mantienen en una base de datos estructurada
3. **Escalabilidad**: Fácil de migrar a bases de datos más robustas (PostgreSQL, MySQL)
4. **Model Registry**: Gestión completa del ciclo de vida de los modelos
5. **Acceso Web**: Interfaz web disponible en http://127.0.0.1:5000

---

## **Casos de Uso Apropiados**

- **Equipos pequeños**: 2-10 científicos de datos
- **Desarrollo local**: Entorno de desarrollo y testing
- **Prototipado**: Validación de conceptos antes de producción
- **Educación**: Cursos y talleres de MLOps
- **Investigación**: Experimentos colaborativos en academia